In [53]:
import pandas as pd
import string
import pymorphy2
from nltk.collocations import BigramAssocMeasures, TrigramAssocMeasures, BigramCollocationFinder
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
import math
from lxml import etree
from nltk.corpus import stopwords
import codecs
import re

Функция преобразует XML-файл корпуса отзывов в обычный текст

In [16]:
def parse_xml(filename):
    with open(filename, encoding='utf-8') as f:
        xml = f.read()

    dict = {}
    text = []
    category = []
    sentiment = []
    term = []

    root = etree.fromstring(xml)
    for child in root:
        for aspect in child[3]:
            if aspect.attrib['type'] == 'implicit':
                text.append(child[2].text)
                category.append(aspect.attrib['category'])
                sentiment.append(aspect.attrib['sentiment'])
                term.append(aspect.attrib['term'])

    dict['text'] = text
    dict['category'] = category
    dict['sentiment'] = sentiment
    dict['term'] = term

    # df = pd.DataFrame(dict)
    # print (df)
    return set(text)


В `all_text` будут хранится все отзывы о ресторанах в виде обычного текста

In [17]:
text_train = parse_xml('SentiRuEval_rest_markup_train.xml')
text_test = parse_xml('SentiRuEval_rest_markup_test.xml')

all_text = ''
for text in text_train:
    all_text += text + ' '

for text in text_test:
    all_text += text + ' ' 

Записала все отзывы о ресторанах в виде обычного текста в файл (пригодится в поиске ключевых слов по TF_IDF)

In [18]:
w = codecs.open('restaurants_corpus.txt' , 'w' , 'utf-8-sig')
w.write(all_text)
w.close()

Следующая функция токенизирует текст, убирает пунктуацию, слова на латинице, стоп-слова, цифры

In [ ]:
def preprocess(text):
    latin = re.compile('[a-zA-Z]+?')
    stop = set(stopwords.words('russian'))
    tmp = word_tokenize(text)
    tokens = []
    for i in tmp:
        if i.isalnum() and i not in stop and not i.isdigit():
            m = re.search(latin, i)
            if m == None:
                tokens.append(i.lower())
    return tokens
 

Предобработка корпуса отзывов о ресторанах

In [54]:
tokens = preprocess(all_text)

Загружаем контрастный корпус  - коллекция новостных статей РИА-новости. Предобработка этого корпуса.

In [106]:
f = open('contrast_corpus.txt' , 'r' , encoding='utf-8-sig')
contrast_corpus = f.read()
contrast_tokens = preprocess(contrast_corpus)

Создаем объект, где хранятся всякие метрики для коллокаций (`bigram_measures`). Также создаем объект, где хранятся все коллокации из ресторанного корпуса (`finder`). В `resto_len` хранится количество всех биграмм ресторанного корпуса

In [108]:
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens)
resto_len = finder.N

То же самое делаем для контрастного корпуса

In [110]:
finder_contrast = BigramCollocationFinder.from_words(contrast_tokens)
contrast_len = finder_contrast.N

Для подсчета IDF понадобится список списков всех биграмм двух корпусов. в `finder.ngram_fd` хранятся только уникальные биграммы, поэтому необходимо дублировать эти биграммы столько раз, сколько они встретились в корпусе

In [127]:
corpus = []
tmp = finder.ngram_fd
resto = []
for w in tmp:
    if tmp[w] > 1:
        n = 0
        while n < tmp[w]:
            resto.append(w)
            n += 1
    else:
        resto.append(w)

tmp2 = finder_contrast.ngram_fd  
contrast = []
for w2 in tmp2:
    if tmp2[w2] > 1:
        n2 = 0
        while n2 < tmp2[w2]:
            contrast.append(w2)
            n2 += 1
    else:
        contrast.append(w2)

corpus.append(resto)
corpus.append(contrast)

Далее, будем смотреть только на биграммы, которые всретились в корпусах более 10 раз. Посмотрим, какие биграммы выделяются в ресторанном корпусе по разным метрикам

In [114]:
finder.apply_freq_filter(10)
finder.nbest(bigram_measures.pmi, 30) 

[('всяких', 'похвал'),
 ('молодым', 'человеком'),
 ('выше', 'всяких'),
 ('первом', 'этаже'),
 ('второй', 'этаж'),
 ('что', 'касается'),
 ('высшем', 'уровне'),
 ('живая', 'музыка'),
 ('молодой', 'человек'),
 ('самое', 'главное'),
 ('приятно', 'удивлены'),
 ('обязательно', 'придем'),
 ('данное', 'заведение'),
 ('день', 'рождения'),
 ('могу', 'сказать'),
 ('добрый', 'день'),
 ('отдельное', 'спасибо'),
 ('хочу', 'отметить'),
 ('большой', 'выбор'),
 ('вкусная', 'еда'),
 ('огромное', 'спасибо'),
 ('долго', 'ждать'),
 ('в', 'общем'),
 ('хочу', 'сказать'),
 ('порции', 'большие'),
 ('в', 'целом'),
 ('уютное', 'место'),
 ('еда', 'вкусная'),
 ('остались', 'довольны'),
 ('большие', 'порции')]

In [115]:
finder.nbest(bigram_measures.chi_sq, 30)

[('всяких', 'похвал'),
 ('молодым', 'человеком'),
 ('что', 'касается'),
 ('выше', 'всяких'),
 ('первом', 'этаже'),
 ('второй', 'этаж'),
 ('день', 'рождения'),
 ('молодой', 'человек'),
 ('живая', 'музыка'),
 ('могу', 'сказать'),
 ('высшем', 'уровне'),
 ('в', 'общем'),
 ('отдельное', 'спасибо'),
 ('приятно', 'удивлены'),
 ('данное', 'заведение'),
 ('обязательно', 'придем'),
 ('самое', 'главное'),
 ('добрый', 'день'),
 ('в', 'целом'),
 ('огромное', 'спасибо'),
 ('большой', 'выбор'),
 ('вкусная', 'еда'),
 ('хочу', 'отметить'),
 ('долго', 'ждать'),
 ('остались', 'довольны'),
 ('хочу', 'сказать'),
 ('еда', 'вкусная'),
 ('порции', 'большие'),
 ('большое', 'спасибо'),
 ('уютное', 'место')]

In [116]:
finder.nbest(bigram_measures.likelihood_ratio, 30)

[('в', 'общем'),
 ('день', 'рождения'),
 ('могу', 'сказать'),
 ('молодой', 'человек'),
 ('в', 'целом'),
 ('отдельное', 'спасибо'),
 ('что', 'касается'),
 ('живая', 'музыка'),
 ('всяких', 'похвал'),
 ('молодым', 'человеком'),
 ('очень', 'довольны'),
 ('огромное', 'спасибо'),
 ('очень', 'вкусно'),
 ('очень', 'понравилось'),
 ('добрый', 'день'),
 ('данное', 'заведение'),
 ('выше', 'всяких'),
 ('остались', 'очень'),
 ('большой', 'выбор'),
 ('второй', 'этаж'),
 ('первом', 'этаже'),
 ('приятно', 'удивлены'),
 ('вкусная', 'еда'),
 ('высшем', 'уровне'),
 ('обязательно', 'придем'),
 ('остались', 'довольны'),
 ('это', 'место'),
 ('долго', 'ждать'),
 ('очень', 'понравился'),
 ('самое', 'главное')]

Применяем тот же фильтр и для контрастного корпуса.

In [117]:
finder_contrast.apply_freq_filter(10)
finder_contrast.nbest(bigram_measures.pmi, 30)

[('торгуемых', 'внебиржевой'),
 ('ла', 'скала'),
 ('непорочного', 'зачатия'),
 ('американскую', 'легкую'),
 ('ролан', 'гаррос'),
 ('мишель', 'бачелет'),
 ('аналогичным', 'периодом'),
 ('сточных', 'вод'),
 ('нури', 'топбаша'),
 ('османа', 'нури'),
 ('подземные', 'толчки'),
 ('континентальной', 'хоккейной'),
 ('промышленных', 'корпораций'),
 ('фондовых', 'дилеров'),
 ('котировок', 'обыкновенных'),
 ('крестный', 'ход'),
 ('поклонной', 'горе'),
 ('божественную', 'литургию'),
 ('курительных', 'смесей'),
 ('крылья', 'советов'),
 ('католических', 'епископов'),
 ('фьючерсов', 'американскую'),
 ('великом', 'новгороде'),
 ('саида', 'нурси'),
 ('первичных', 'заявок'),
 ('крепость', 'мусульманина'),
 ('северного', 'кавказа'),
 ('фондовые', 'индексы'),
 ('бюджетного', 'дефицита'),
 ('массовых', 'коммуникаций')]

В `bigrams_resto` хранится список уникальных биграммов, встретившихся в ресторанном корпусе более 10 раз + их частотность

In [118]:
bigrams_resto = finder.ngram_fd

В `bigrams_contrast` хранится список уникальных биграммов, встретившихся в контрастном корпусе более 10 раз + их частотность

In [119]:
bigrams_contrast = finder_contrast.ngram_fd

Подсчитаем TF-IDF для биграмм ресторанного корпуса

In [131]:
def compute_tf(fd, textlen):
    res = {}
    for i in fd:
        res[i] = fd[i]/textlen
    return res

def compute_idf(ngram, corpus):
    return math.log10(len(corpus)/sum([1.0 for text in corpus if ngram in text]))

tf_resto = compute_tf(bigrams_resto, resto_len)

tf_idf_resto = {}
for i in tf_resto:
    tf_idf_value = tf_resto[i] * compute_idf(i, corpus)
    if tf_idf_value not in tf_idf_resto.keys():
        x = [i]
        tf_idf_resto[tf_idf_value] = x
    else:
        tf_idf_resto[tf_idf_value].append(i)

In [134]:
for result in sorted(tf_idf_resto, reverse=True):
    print ('score: ' + str(result))
    for i in tf_idf_resto[result]:
        print (i)
    print('---------------------------')

score: 0.0005241221135331092
('в', 'общем')
---------------------------
score: 0.000422975740746018
('очень', 'вкусно')
---------------------------
score: 0.0004137806159471915
('очень', 'понравилось')
---------------------------
score: 0.0003310244927577532
('очень', 'довольны')
---------------------------
score: 0.00030343911836127376
('остались', 'очень')
('могу', 'сказать')
---------------------------
score: 0.00024826836956831486
('это', 'место')
---------------------------
score: 0.0002390732447694884
('молодой', 'человек')
('очень', 'понравился')
---------------------------
score: 0.00022987811997066191
('отдельное', 'спасибо')
---------------------------
score: 0.00019309762077535602
('огромное', 'спасибо')
('очень', 'вкусные')
---------------------------
score: 0.0001655122463788766
('это', 'заведение')
('нам', 'очень')
('очень', 'долго')
('живая', 'музыка')
---------------------------
score: 0.00015631712158005012
('очень', 'приятно')
('очень', 'быстро')
---------------------

In [52]:
trigram_measures = TrigramAssocMeasures()
finder1 = TrigramCollocationFinder.from_words(tokens)
finder1.apply_freq_filter(5)
finder1 = TrigramCollocationFinder.from_words(tokens, window_size=3)
finder1.nbest(trigram_measures.pmi, 30)

[('le', 'glamure', 'хрустальный'),
 ('mon', 'petit', 'cafe'),
 ('sokos', 'hotel', 'vasilievsky'),
 ('антонина', 'разбирается', 'предсталенной'),
 ('ассорти', 'капусту', 'тушенную'),
 ('бадене', 'живём', 'озерках'),
 ('базар', 'перед', 'уходом'),
 ('беду', 'решаю', 'отвести'),
 ('бекон', 'хрустящем', 'тосте'),
 ('беседки', 'веселые', 'фигурки'),
 ('бесплатной', 'воды', 'бокалам'),
 ('бешеную', 'популярность', 'владельцы'),
 ('быстрого', 'ненавязчивого', 'доброжелательного'),
 ('вернее', 'месторасполажения', 'приличных'),
 ('вес', 'картинках', 'речи'),
 ('веселую', 'программу', 'аниматоры'),
 ('видны', 'китайские', 'аттрибуты'),
 ('включала', 'грецкие', 'орехи'),
 ('воздушные', 'шары', 'требуются'),
 ('возникла', 'картина', 'воспоминаний'),
 ('восточному', 'инжирным', 'вареньем'),
 ('встречаются', 'зазвездившиеся', 'админы'),
 ('вторую', 'смену', 'обидел'),
 ('выполнения', 'церемонии', 'чаяналивания'),
 ('выступали', 'egeiro', 'project'),
 ('высшее', 'наслаждение', 'пробуйте'),
 ('выходи

In [50]:
finder1.nbest(trigram_measures.chi_sq, 30)

[('le', 'glamure', 'хрустальный'),
 ('mon', 'petit', 'cafe'),
 ('sokos', 'hotel', 'vasilievsky'),
 ('антонина', 'разбирается', 'предсталенной'),
 ('ассорти', 'капусту', 'тушенную'),
 ('бадене', 'живём', 'озерках'),
 ('базар', 'перед', 'уходом'),
 ('беду', 'решаю', 'отвести'),
 ('бекон', 'хрустящем', 'тосте'),
 ('беседки', 'веселые', 'фигурки'),
 ('бесплатной', 'воды', 'бокалам'),
 ('бешеную', 'популярность', 'владельцы'),
 ('быстрого', 'ненавязчивого', 'доброжелательного'),
 ('вернее', 'месторасполажения', 'приличных'),
 ('вес', 'картинках', 'речи'),
 ('веселую', 'программу', 'аниматоры'),
 ('видны', 'китайские', 'аттрибуты'),
 ('включала', 'грецкие', 'орехи'),
 ('воздушные', 'шары', 'требуются'),
 ('возникла', 'картина', 'воспоминаний'),
 ('восточному', 'инжирным', 'вареньем'),
 ('встречаются', 'зазвездившиеся', 'админы'),
 ('вторую', 'смену', 'обидел'),
 ('выполнения', 'церемонии', 'чаяналивания'),
 ('выступали', 'egeiro', 'project'),
 ('высшее', 'наслаждение', 'пробуйте'),
 ('выходи

In [53]:
finder1.nbest(trigram_measures.likelihood_ratio, 30)

[('остались', 'очень', 'довольны'),
 ('в', 'общем', 'целом'),
 ('в', 'общем', 'остались'),
 ('дорого', 'в', 'общем'),
 ('в', 'общем', 'прекрасно'),
 ('в', 'общем', 'придраться'),
 ('в', 'общем', 'вечер'),
 ('бесплатная', 'в', 'общем'),
 ('в', 'общем', 'зайдём'),
 ('в', 'общем', 'оплатили'),
 ('в', 'общем', 'порекомендую'),
 ('ведущий', 'в', 'общем'),
 ('заметна', 'в', 'общем'),
 ('заявлен', 'в', 'общем'),
 ('комки', 'в', 'общем'),
 ('лезло', 'в', 'общем'),
 ('лучшем', 'в', 'общем'),
 ('одних', 'в', 'общем'),
 ('поднебесной', 'в', 'общем'),
 ('прочерк', 'в', 'общем'),
 ('развились', 'в', 'общем'),
 ('телефончик', 'в', 'общем'),
 ('в', 'общем', 'это'),
 ('в', 'общем', 'впечатления'),
 ('вкусный', 'в', 'общем'),
 ('кальян', 'в', 'общем'),
 ('голос', 'в', 'общем'),
 ('ожидании', 'в', 'общем'),
 ('фамилии', 'в', 'общем'),
 ('в', 'общем', 'очень')]